# MIE 1613 A3 
- Due Date: 2023-03-22
- Name: William Hazen
- Student ID: 1009231225
- Department: MIE
- Program: M.Eng.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import math
from scipy.stats import probplot
from scipy.stats import norm
warnings.filterwarnings("ignore")
# fix random number seed
np.random.seed(1)

In [1]:
import SimRNG
import SimClasses
import SimFunctions

# **1.**

### **1a)**

In [ ]:
semiconductordata_df = pd.read_excel("SemiconductorData.xls")
semiconductordata_df.head()

In [ ]:
new_SCD_df = semiconductordata_df

In [ ]:
new_SCD_df["LOAD/UNLOAD"] = semiconductordata_df.iloc[:, 1]

In [ ]:
new_SCD_df.iloc[0]


### **1b)**

# **2.**

In [ ]:

def Drone_Sim (s):
    ZSimRNG = SimRNG.InitializeRNSeed()

    Queue = SimClasses.FIFOQueue()
    Wait = SimClasses.DTStat()
    Server = SimClasses.Resource()
    Calendar = SimClasses.EventCalendar()

    TheCTStats = []
    TheDTStats = []
    TheQueues = []
    TheResources = []

    TheDTStats.append(Wait)
    TheQueues.append(Queue)
    TheResources.append(Server)

    Server.SetUnits(s)  # changed SetUnits(1) to SetUnits(s)
    MeanTBA = 1.0
    MeanDroneRate = 3/4  # 45min
    RunLength = 24*60

    # WarmUp = #5000.0

    AllWaitMean = []
    AllQueueMean = []
    AllQueueNum = []
    AllServerMean = []

    def Call_Rate():

        if SimClasses.Clock < 6:
            call_rate = 4  # 7am-12pm
        elif SimClasses.Clock < 13:
            call_rate = 2  # 12pm-12am
        else:
            call_rate = 1  # 12am-7am

        return call_rate

    def CI_95(data):    # define 95% confidence interval function
        a = np.array(data)
        n = len(a)
        m = np.mean(a)
        sd = np.std(a, ddof=1)
        hw = 1.96*sd / np.sqrt(n)
        return m, m-hw, m+hw

    def Arrival():
        SimFunctions.Schedule(Calendar, "Arrival", SimRNG.Expon(MeanTBA, 1))
        Customer = SimClasses.Entity()
        Queue.Add(Customer)

        if Server.Busy < s:
            if Server.Busy >= 0:
                Server.Seize(1)
                SimFunctions.Schedule(
                    Calendar, "EndOfService", SimRNG.Expon(MeanDroneRate, 2))
        else:
            pass

    def EndOfService():
        DepartingCustomer = Queue.Remove()
        if DepartingCustomer is not None:
            Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
            # modified EndOfService function to account for more than one server
            # We only record a time if there is a DepartingCustomer
        else:
            pass

        # if Queue.NumQueue() >= s:
        #     SimFunctions.Schedule(Calendar, "EndOfService",
        #                           SimRNG.Erlang(Phases, MeanDroneRate, 2))
        # else:
        #     Server.Free(1)


    for reps in range(0, 1000, 1):  # 1000 repetitions

        SimFunctions.SimFunctionsInit(Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
        SimFunctions.Schedule(Calendar, "Arrival", SimRNG.Expon(MeanTBA, 1))
        SimFunctions.Schedule(Calendar, "EndSimulation", RunLength)
        # SimFunctions.Schedule(Calendar, "ClearIt", WarmUp)

        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        if NextEvent.EventType == "Arrival":
            Arrival()
        elif NextEvent.EventType == "EndOfService":
            EndOfService()
        # elif NextEvent.EventType == "ClearIt":
        #     SimFunctions.ClearStats(TheCTStats, TheDTStats)

        while NextEvent.EventType != "EndSimulation":
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime
            if NextEvent.EventType == "Arrival":
                Arrival()
            elif NextEvent.EventType == "EndOfService":
                EndOfService()
            # elif NextEvent.EventType == "ClearIt":
            #     SimFunctions.ClearStats(TheCTStats, TheDTStats)

        AllWaitMean.append(Wait.Mean())
        AllQueueMean.append(Queue.Mean())
        AllQueueNum.append(Queue.NumQueue())
        AllServerMean.append(Server.Mean())

        output = {"AllWaitMean": AllWaitMean, "CI_95 (-)": CI_95(AllWaitMean)[1], "CI_95 (+)": CI_95(AllWaitMean)[2], "AllQueueMean": np.mean(AllQueueMean), "AllServerMean:": np.mean(AllServerMean)}
    return pd.DataFrame(output)


### **2a)**

- The model I choose to estimate the minimum number of drones that will be readliy available at the event of a cardiac arrest is the MGS queue. This decision was choosen based on the model following a queuing system with an arrival rate that is assumed to be non-homogeneous Possion process.

### **2b)**

In [ ]:
d = 1000
df_2a = Drone_Sim(d)

In [ ]:
df_2a

# **3.**

### **3a)**

In [ ]:
TSLA_df = pd.read_csv("TSLA.csv")
TSLA_df.head()

In [ ]:
TSLA_df.Date.sort_values(ascending=True)

In [ ]:
TSLA_df


In [ ]:
TSLA_df["log_returns"] = np.log(TSLA_df['Close'] / TSLA_df['Close'].shift(1))


In [ ]:
TSLA_df["log_returns"] 
TSLA_df["log_returns"]


In [ ]:
tesla_mu = TSLA_df["log_returns"].mean()
tesla_sigma = TSLA_df["log_returns"].std(ddof=1)

print(tesla_mu, tesla_sigma)


In [ ]:
sns.displot(TSLA_df["Close"])
plt.title("TSLA Close")
plt.show()

sns.displot(TSLA_df["log_returns"])
plt.title("TSLA Log Return")
plt.show()

In [ ]:
np.random.seed(1)

n = len(TSLA_df)
log_returns_sim = np.random.normal(tesla_mu, tesla_sigma, n)

# Simulate future stock prices using GBM model
S1 = TSLA_df['Close'].iloc[-1]  
Sim_TSLA = []
Sim_TSLA.append(S1)
for i in range(1, n):
    Sim_TSLA.append(Sim_TSLA[i-1] * np.exp(log_returns_sim[i]))

plt.figure(figsize=(10,6))
plt.plot(TSLA_df['Close'])
plt.plot(Sim_TSLA)
plt.legend(['Actual', 'Simulated'])
plt.title("Simulated TSLA Close Price")
plt.xlabel("n")
plt.ylabel("Close Price")
plt.show()


In [ ]:
sns.displot(Sim_TSLA)
plt.title("Simulated TSLA")
plt.show()


### **3b)**

In [ ]:
figure = plt.figure(figsize=(20, 5))
ax1 = figure.add_subplot(131)
ax1 = probplot(TSLA_df["Close"], plot=plt)
plt.title("TSLA Close")

ax2 = figure.add_subplot(132)
ax2 = probplot(TSLA_df["log_returns"], plot=plt)
plt.title("TSLA Log Returns")

ax3 = figure.add_subplot(133)
ax3 = probplot(Sim_TSLA, plot=plt)
plt.title("Simulated Tesla")
plt.show()


# **4.**

### **4a)**

In [ ]:
x_0 = 0
gamma = 2
x_list = []
x = 0

np.random.seed(3)
for i in range(1000):
    # Fx = 1/np.pi * np.arctan(np.random.uniform() - x_0/ gamma) + 1/2
    x = gamma * np.tan((np.random.uniform(0, 1) - 0.5) * np.pi) + x_0
    x_list.append(x)

In [ ]:
x_list_plot = sns.displot(x_list)
x_list_plot.fig.set_figwidth(15)
x_list_plot.fig.set_figheight(6)
plt.show()

### **4b)**

In [ ]:
sorted_x_list = np.sort(x_list)

half_quantile = sorted_x_list[int(1/2 * len(sorted_x_list))]
inter_quantile = sorted_x_list[int(3/4 * len(sorted_x_list))] - sorted_x_list[int(1/4 * len(sorted_x_list))]

print(half_quantile, inter_quantile)


# **5.**

In [ ]:
CallCounts_df = pd.read_excel("CallCounts.xls")
CallCounts_df.head()

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
mean_calls = CallCounts_df.describe()[1:2]
std_calls = CallCounts_df.describe()[2:3]

In [ ]:
mean_calls = mean_calls.values[0]
mean_calls


In [ ]:
var_calls = std_calls.values[0]**2
var_calls

In [ ]:
mean_call_list = []
mean_cum_sum = 0
for call in mean_calls:
    mean_cum_sum += call
    mean_call_list.append(mean_cum_sum)

In [ ]:
mean_call_list

In [ ]:
var_call_list = []
var_cum_sum = 0
for call in var_calls:
    var_cum_sum += call
    var_call_list.append(var_cum_sum)


In [ ]:
var_call_list

In [ ]:
X = np.array(mean_call_list).reshape(-1,1)
y = var_call_list

model = LinearRegression()
model.fit(X,y)
predictions = model.predict(X)
score = model.score(X,y)

In [ ]:
plt.scatter(X, y)
plt.plot(X, predictions, color='red')
plt.title("Linear Regression of Call Data")
plt.xlabel("Mean Call")
plt.ylabel("Var Call")
plt.show()


In [ ]:
results = pd.DataFrame( {"Cumlative Sum": mean_call_list, "Regression Results": predictions})
results

In [ ]:
print('Beta:', model.coef_[0])
print("R^2:", score)